In [1]:
import pyvista as pv
from src.lddmm.registrator import LDDMMRegistrator

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


In [2]:
source_mesh = pv.read("/Users/patrykrygiel/Documents/UTWENTE/Datasets/AAA/BAEK/meshes/P20_20080212_0000/full.vtp").decimate(0.95)
target_mesh = pv.read("/Users/patrykrygiel/Documents/UTWENTE/Datasets/AAA/BAEK/meshes/P20_20110310_0000/full.vtp").decimate(0.95)

source_mesh.points -= source_mesh.points.mean(axis=0)
target_mesh.points -= target_mesh.points.mean(axis=0)

print(source_mesh.points.shape)
print(target_mesh.points.shape)

pl = pv.Plotter()
pl.add_mesh(source_mesh, color="green", opacity=0.5)
pl.add_mesh(target_mesh, color="red", opacity=0.5)
pl.show()

(2838, 3)
(3526, 3)


Widget(value='<iframe src="http://localhost:62523/index.html?ui=P_0x2d3d42010_0&reconnect=auto" class="pyvista…

In [3]:
registrator = LDDMMRegistrator(source_mesh, target_mesh, sigma=20, device="cpu")
registrator.optimize(2)
registrator.export_shoot("shots", nt=15, deltat=1.5)

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
registrator.export_shoot("shots", nt=30, deltat=3.0)